In [6]:
#imports
import os
import re
import time
import pandas as pd
from IPython.display import display
from IPython.display import HTML
from datetime import datetime, timezone
import csv

base_pathfile = r'C:\Users\Jesus.Ruiz\OneDrive - AECOM\Documents\test\test.csv' #placeholder
base_dir = r'C:\Users\Jesus.Ruiz\OneDrive - AECOM\Documents\test\demo_RBP' #placeholder
base_paths = [] #placeholder

def csvDf(base_pathfile):
    df = pd.read_csv(base_pathfile)
    return df

mydf = csvDf(base_pathfile)
base_paths = mydf['Files'].tolist()

#index_ =mydf.set_index('Files')
#mydf.index = index_
#indices = mydf.index.tolist()

base_paths = mydf['Files'].tolist()

result = mydf.loc[mydf['Files']=="Project2.rvt",'Size'].item()
print(result)
#base_size = mydf['Size'].tolist()


def formatSize(sb):
    mb = 1048576.0
    kb = 1024.0
    if abs(sb) < mb:
        return '%.3f KB' % (sb / kb)
    else:
        return '%.3f MB' % (sb / mb)

def get_nwc_files(base_dir):
    # to initialize multiple lists
    Files, Date_Modified,Base_Size, Size, Directory = ([] for i in range(5))
    for entry in os.scandir(base_dir):
        if entry.is_file() and entry.name.endswith('.rvt') and entry.name in base_paths:
            Files.append(entry.name)
            #Check if file is new
            print(entry.name) 
            isNew = time.time() - entry.stat().st_mtime < (3*24*60*60)
            print(isNew)
            #Compare file sizes
            base_size_notation = mydf.loc[mydf['Files']==entry.name,'Size'].item()                            
            base_size_string = re.findall("\d+\.\d+", base_size_notation)[0]
            if base_size_notation.endswith('MB'):
                base_size = float(base_size_string) * 1048576.0
            else:
                base_size = float(base_size_string) * 1024.0
            new_size = entry.stat().st_size
            dif_size_bytes = new_size - base_size
            dif_size = formatSize(dif_size_bytes)
            print (dif_size)
            Date_Modified.append(datetime.fromtimestamp(entry.stat().st_mtime).strftime('%Y-%m-%d-%H:%M'))
            Size.append(formatSize(entry.stat().st_size))
            Directory.append(entry.path)
        if entry.is_dir():
            get_nwc_files(entry.path)
        else:
            pass
            #print(f"Neither a file, nor a dir: {entry.path}")

    df = pd.DataFrame({'Files': Files, 'Date Modified': Date_Modified,'Size':Size,'Directory': Directory})      
    return df

def highligh_rows(row):
    value = row.loc['Date Modified']
    base_size_string = re.findall("\d+\.\d+", value)[0]
    if value.endswith('MB'):
        base_size = float(base_size_string) * 1048576.0
    else:
        base_size = float(base_size_string) * 1024.0
    isNew = time.time() - base_size <   (3*24*60*60)
    if not isNew:
        color = '#FFB3BA' # Red
    return ['background-color: {}'.format(color) for r in row]


df = get_nwc_files(base_dir)
df.style.apply(highligh_rows,axis=1)

def highlight_col(x):
    #copy df to new - original data are not changed
    df = x.copy()
    #set by condition
    mask = df['Files'] == 'Project1.rvt'
    df.loc[mask, :] = 'background-color: yellow'
    df.loc[~mask,:] = 'background-color: ""'
    return df   

df.style.apply(highlight_col, axis=None)
print(df)

display(df)

""" for item in os.scandir(base_dir):
    print (item.name,
            datetime.fromtimestamp(item.stat().st_mtime).strftime('%Y-%m-%d-%H:%M'),
            formatSize(item.stat().st_size),
            item.path) """



1.00 MB
Project1.rvt
False
-46.523 MB
Project2.rvt
False
2.477 MB
          Files     Date Modified      Size  \
0  Project1.rvt  2022-06-02-10:27  3.477 MB   
1  Project2.rvt  2022-06-02-10:27  3.477 MB   

                                           Directory  
0  C:\Users\Jesus.Ruiz\OneDrive - AECOM\Documents...  
1  C:\Users\Jesus.Ruiz\OneDrive - AECOM\Documents...  


,Files,Date Modified,Size,Directory
0,Project1.rvt,2022-06-02-10:27,3.477 MB,C:\Users\Jesus.Ruiz\OneDrive - AECOM\Documents...
1,Project2.rvt,2022-06-02-10:27,3.477 MB,C:\Users\Jesus.Ruiz\OneDrive - AECOM\Documents...


" for item in os.scandir(base_dir):\n    print (item.name,\n            datetime.fromtimestamp(item.stat().st_mtime).strftime('%Y-%m-%d-%H:%M'),\n            formatSize(item.stat().st_size),\n            item.path) "